# Importando e pré-processando os dados

In [2]:
import pandas as pd
import glob
import numpy as np

Os dados dessa base, costumam serem apresentados de forma mensal, fazendo-se necessário sua concatenação prévia para posterior análise dos dados. Para este trabalho, foram usados dados de Janeiro de 2018 até agosto de 2021

Para este documento trabalharemos com a base já concatenada, no entanto, caso haja necessidade há uma função pronta neste diretório do github, podendo ser acessada via:

- from functions import concatenador

In [5]:
df=pd.read_csv(r'C:\Users\efvs\Documents\Residencia Ednael\Repositórios\data_all.zip',compression='zip')

In [6]:
df.head()

,Unnamed: 0,Número Licitação,Código UG,Nome UG,Código Modalidade Compra,Modalidade Compra,Número Processo,Objeto,Situação Licitação,Código Órgão Superior,Nome Órgão Superior,Código Órgão,Nome Órgão,Município,Data Resultado Compra,Data Abertura,Total
0,0,12018,120630,BASE AEREA DE DE MANAUS,3,Concorrência,67290015424201435,Objeto: Contratação de empresa de engenharia e...,Evento de Resultado de Julgame,52000,Ministério da Defesa,52111,Comando da Aeronáutica,MANAUS,19/07/2018,NaN,1063307.91
1,1,12018,153015,CENTRO FEDERAL DE EDUCACAO TECNOLOGICA DE MG,3,Concorrência,23062002056/18-26,Objeto: A presente Licitação tem como objeto a...,Evento de Resultado de Julgame,26000,Ministério da Educação,26257,Centro Federal de Educação Tecnológica de Min,BELO HORIZONTE,19/07/2018,NaN,689750.64
2,2,12018,153036,UNIVERSIDADE FED.VALES JEQUITINHONHA E MUCURI,-99,Pregão - Registro de Preço,23086000649201861,Objeto: Pregão Eletrônico - Sistema de Regist...,Evento de Suspensão Publicado,26000,Ministério da Educação,26255,Universidade Federal dos Vales do Jequitinhon,DIAMANTINA,12/07/2018,19/06/2018,714.57
3,3,12018,160141,COMISSAO REGIONAL DE OBRAS DA 9ª RM,3,Concorrência,65328001557201858,Objeto: OBRA DE CONSTRUÇÃO DO PAVILHÃO COMPANH...,Evento de Resultado de Julgame,52000,Ministério da Defesa,52121,Comando do Exército,CAMPO GRANDE,23/07/2018,NaN,3241361.87
4,4,12018,200130,SUPERINTENDENCIA REG. POL. RODV. FEDERAL-SE,3,Concorrência,08672001011/2017-,Objeto: Contratação de empresa de engenharia p...,Evento de Alteração Publicado,30000,Ministério da Justiça e Segurança Pública,30802,Departamento de Polícia Rodoviária Federal,ARACAJU,18/07/2018,NaN,3222277.60


# Feature Engineering

- Adding information about the licitation type
- Removing duplicated
- Encoding

In [70]:
import pandas as pd

def importa_dados_zip(url):
    
    # url = link para o local onde os dados estão armazenados
    df = pd.read_csv(url,compression='zip')
    
    # Deletando/criando coluna desnecessaria
    del(df['Unnamed: 0'])
    df['Tag'] = 0 #serah usado posteriormente, na fase de taguiamento
    
    # Ajustando nomes
    df.rename(columns={'Data Resultado Compra':'data'},inplace=True)
    
    return (df)

In [71]:
df = importa_dados_zip(r'C:\Users\efvs\Documents\Residencia Ednael\Repositórios\data_all.zip')

In [98]:
r'C:\Users\efvs\Documents\Residencia Ednael\Repositórios\data_all.zip'

'C:\\Users\\efvs\\Documents\\Residencia Ednael\\Repositórios\\data_all.zip'

In [29]:
def define_obra(data):
    
    result = data.Objeto.str.contains("OBRAS|OBRA|obras|obra|ENGENHARIA|engenharia|CONSTRU-CAO|constru-cao", na=False)
    
    return (result.replace({True:'Obra',False:'Compras/Servicos'}))

In [73]:
df['Obras'] = define_obra(df)

First classification - Based on law <i>`Decreto nº 9.412, de 18 de Junho de 2018`

In [108]:
def classificador(aa):
    
    #dados no formato Numpy
    
    for linhas in aa:

        #TAG'AMENTO DAS LICITACOES DDO TIPO: REGISTRO DE PRECO
        if (linhas[4]=='Pregão - Registro de Preço' or linhas[4]=='Concorrência - Registro de Preço' or linhas[4]=='Concorrência')  and linhas[-3]>3300000 and linhas[-2]=='Obras':
            linhas[-1] = 1 #EH LEGAL
        if(linhas[4]=='Pregão - Registro de Preço')  and (linhas[-3] > 1430000) and (linhas[-2]=='Compras/Servicos'):
            linhas[-1] = 1 #EH LEGAL
            
            
        #TAG'AMENTO DAS LICITACOES DO TIPO: DISPENSA LICITACAO
        if(linhas[4]=='Dispensa de Licitação')  and (linhas[-3] <= 33000) and (linhas[-2]=='Obras'):
            linhas[-1] = 1 #EH LEGAL
        if(linhas[4]=='Dispensa de Licitação')  and (linhas[-3] <= 17600) and (linhas[-2]=='Compras/Servicos'):
            linhas[-1] = 1 #EH LEGAL
            
            
        #TAG'AMENTO DAS LICITACOES DO TIPO: TOMADA DE PREÇOS
        if(linhas[4]=='Tomada de Preços')  and (linhas[-3] > 330000 and linhas[-3] <= 3300000) and (linhas[-2]=='Obras'):
            linhas[-1] = 1
        if(linhas[4]=='Tomada de Preços')  and (linhas[-3] > 176000 and linhas[-3] <= 1400000) and (linhas[-2]=='Compras/Servicos'):
            linhas[-1] = 1 #EH LEGAL
            
        #TAG'AMENTO DAS LICITACOES DO TIPO: CONVITE
        if(linhas[4]=='Convite')  and (linhas[-3] > 17600 and linhas[-3] <= 176000) and (linhas[-2]=='Obras'):
            linhas[-1] = 1 #EH LEGAL
        if(linhas[4]=='Convite')  and (linhas[-3] > 33000 and linhas[-3] <= 330000) and (linhas[-2]=='Compras/Servicos'):
            linhas[-1] = 1 #EH LEGAL
            
        #TAG'AMENTO DAS LICITACOES DO TIPO: PREGÃO | INEXIGIBILIDADE DE LICITAÇÃO | CONCURSO | CONCORRENCIA INTERNACIONAL
        if(linhas[4]=='Pregão' or linhas[4]=='Inexigibilidade de Licitação' or linhas[4]=='Concurso' or linhas[4]=='Concorrência Internacional'):
            linhas[-1]=1 #EH LEGAL
    
    return(aa)

In [111]:
df_rotulado = pd.DataFrame(classificador(df2),columns=colunas)

In [95]:
df.columns

Index(['Número Licitação', 'Código UG', 'Nome UG', 'Código Modalidade Compra',
       'Modalidade Compra', 'Número Processo', 'Objeto', 'Situação Licitação',
       'Código Órgão Superior', 'Nome Órgão Superior', 'Código Órgão',
       'Nome Órgão', 'Município', 'data', 'Data Abertura', 'Total', 'Tag',
       'Obras'],
      dtype='object')

In [114]:
df_rotulado['Tag'].value_counts()

1    406195
0     94011
Name: Tag, dtype: int64

In [115]:
df_rotulado.to_csv('../data/dados_taguiados.csv')

In [ ]:
import framework_00

In [2]:
from framework_00 import df

In [52]:
def calcula_pct_teto(data):
    
    # Inicialização
    data = np.array(data)
    pct_teto = []
    ###############
    for linhas in data:
        
        if linhas[-1] == 0:
            #print(linhas)
            #TAG'AMENTO DAS LICITACOES DDO TIPO: REGISTRO DE PRECO
            if (linhas[4]=='Pregão - Registro de Preço' or linhas[4]=='Concorrência - Registro de Preço' or linhas[4]=='Concorrência') and (linhas[-2]=='Obras'):
                pct_teto.append(linhas[-3] / 3300000)

            if (linhas[4]=='Pregão - Registro de Preço') and (linhas[-2]=='Compras/Servicos'):
                pct_teto.append(linhas[-3] / 1430000)

            #TAG'AMENTO DAS LICITACOES DO TIPO: DISPENSA LICITACAO
            if (linhas[4]=='Dispensa de Licitação') and (linhas[-2]=='Obras'):
                pct_teto.append(linhas[-3] / 33000)
            if (linhas[4]=='Dispensa de Licitação')  and (linhas[-2]=='Compras/Servicos'):
                pct_teto.append(linhas[-3] / 17600)

            #TAG'AMENTO DAS LICITACOES DO TIPO: TOMADA DE PREÇOS
            if (linhas[4]=='Tomada de Preços')  and (linhas[-2]=='Obras'):
                pct_teto.append(linhas[-3] / 3300000)                          
            if (linhas[4]=='Tomada de Preços')  and (linhas[-2]=='Compras/Servicos'):
                pct_teto.append(linhas[-3] / 1400000)

            #TAG'AMENTO DAS LICITACOES DO TIPO: CONVITE
            if (linhas[4]=='Convite')  and (linhas[-2]=='Obras'):
                pct_teto.append(linhas[-3] / 176000)
            if (linhas[4]=='Convite')   and (linhas[-2]=='Compras/Servicos'):
                pct_teto.append(linhas[-3] / 330000)

    return(pct_teto)

In [53]:
pcts_teto = calcula_pct_teto(df)

In [54]:
import pandas as pd

pcts_teto = pd.DataFrame(pcts_teto)
pcts_teto.describe()                  

,0
count,90768.000000
mean,31.288002
std,789.157751
min,0.000000
25%,0.000925
50%,0.020205
75%,1.534091
max,88011.575877


Por algum motivo, dando diferença entre essas 2 quantidades

In [75]:
df[df['Tag']==0].shape

(99845, 18)

In [72]:
pcts_teto.shape

(90768, 1)

In [70]:
df3 = pd.merge([df,pcts_teto],right=True)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [57]:
df3

,0
0,Número Licitação Código UG \ 0 ...
1,0 0 0.003413 1 0.00042...


In [77]:
from framework_00 import df

In [78]:
df

,Número Licitação,Código UG,Nome UG,Código Modalidade Compra,Modalidade Compra,Número Processo,Objeto,Situação Licitação,Código Órgão Superior,Nome Órgão Superior,Código Órgão,Nome Órgão,Município,data,Data Abertura,Total,Obras,Tag
0,12018,120630,BASE AEREA DE DE MANAUS,3,Concorrência,67290015424201435,Objeto: Contratação de empresa de engenharia e...,Evento de Resultado de Julgame,52000,Ministério da Defesa,52111,Comando da Aeronáutica,MANAUS,19/07/2018,NaN,1063307.91,Obra,0
1,12018,153015,CENTRO FEDERAL DE EDUCACAO TECNOLOGICA DE MG,3,Concorrência,23062002056/18-26,Objeto: A presente Licitação tem como objeto a...,Evento de Resultado de Julgame,26000,Ministério da Educação,26257,Centro Federal de Educação Tecnológica de Min,BELO HORIZONTE,19/07/2018,NaN,689750.64,Obra,0
2,12018,153036,UNIVERSIDADE FED.VALES JEQUITINHONHA E MUCURI,-99,Pregão - Registro de Preço,23086000649201861,Objeto: Pregão Eletrônico - Sistema de Regist...,Evento de Suspensão Publicado,26000,Ministério da Educação,26255,Universidade Federal dos Vales do Jequitinhon,DIAMANTINA,12/07/2018,19/06/2018,714.57,Obra,0
3,12018,160141,COMISSAO REGIONAL DE OBRAS DA 9ª RM,3,Concorrência,65328001557201858,Objeto: OBRA DE CONSTRUÇÃO DO PAVILHÃO COMPANH...,Evento de Resultado de Julgame,52000,Ministério da Defesa,52121,Comando do Exército,CAMPO GRANDE,23/07/2018,NaN,3241361.87,Obra,0
4,12018,200130,SUPERINTENDENCIA REG. POL. RODV. FEDERAL-SE,3,Concorrência,08672001011/2017-,Objeto: Contratação de empresa de engenharia p...,Evento de Alteração Publicado,30000,Ministério da Justiça e Segurança Pública,30802,Departamento de Polícia Rodoviária Federal,ARACAJU,18/07/2018,NaN,3222277.6,Obra,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500201,954632021,153035,UNIVERSIDADE FEDERAL DO TRIANGULO MINEIRO,7,Inexigibilidade de Licitação,23085005463/21-11,Objeto: Pagamento de registro e emissão de DOI...,Encerrado,26000,Ministério da Educação,26254,Universidade Federal do Triângulo Mineiro,UBERABA,01/07/2021,NaN,3672.0,Compras/Servicos,1
500202,12020,344002,FUNDACAO JOAQUIM NABUCO,20,Concurso,000426/2020-35,Objeto: Constitui objeto do VI Concurso de Res...,Publicado,26000,Ministério da Educação,26292,Fundação Joaquim Nabuco,RECIFE,31/05/2021,NaN,45000.0,Obra,1
500203,22020,154069,FUNDACAO UNIVERSIDADE FEDERAL DE S.J.DEL-REI,20,Concurso,23122.013028/2020,Objeto: Realização do 15º Concurso de Presépio...,Publicado,26000,Ministério da Educação,26285,Fundação Universidade Federal de São João Del,SAO JOAO DEL REI,15/01/2021,NaN,4300.0,Compras/Servicos,1
500204,12021,183038,INSTITUTO NACIONAL DA PROPRIEDADE INDUSTRIAL,20,Concurso,52402008110202042,Objeto: Evento online HACKATHON INPI n 01/2021...,Evento de Resultado de Julgame,25000,Ministério da Economia,30204,Instituto Nacional da Propriedade Industrial,RIO DE JANEIRO,30/04/2021,NaN,19490.54,Compras/Servicos,1
